## Overview ##

PubChem is a site run by the NIH which hosts raw data associated with chemical experiments; here we analyze the data hosted at PubChem for assay 1030, which looks for inhibitors of the protein encoding gene ALDH1A1. You can access the page for this assay [here](https://pubchem.ncbi.nlm.nih.gov/bioassay/1030)

## Results ##

We use the SMILES string, a common representation for a molecule amongst chemists, to begin the featurization process. Because the length of this string varies, it is normalized in the form of a Morgan Fingerprint; these are then used to train various DNN-based classifiers in this notebook

In [1]:
# Exploratory data analysis and visualization

In [2]:
import pickle
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, rdMolDescriptors
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.model_selection import train_test_split
import sys

import warnings
warnings.filterwarnings('ignore')

global_random_state = 42

np.random.seed(global_random_state)


active_pct = 0.073125471
inactive_pct = 1 - active_pct


import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

fh = logging.FileHandler('log_dnn.txt')
fh.setLevel(logging.INFO)
fh.setFormatter(formatter)
logger.addHandler(fh)

ch = logging.StreamHandler(sys.stdout)
ch.setLevel(logging.INFO)
ch.setFormatter(formatter)
logger.addHandler(ch)

In [3]:
# and downloading the datatable
from imblearn.under_sampling import RandomUnderSampler

ba_df = pd.read_csv("AID_1030_datatable_all.csv")

# Load compound info
cs_df = pd.read_csv("AID_1030_compound_smiles.csv",sep='\t',header=0)

# Merge the two
full_df = ba_df.merge(cs_df,on='PUBCHEM_CID')

# Cleanup the compound ID column
full_df["PUBCHEM_CID"] = full_df["PUBCHEM_CID"].astype(int)

# Delete CID 3246048, which fails featurization
full_df = full_df[full_df["PUBCHEM_CID"] != 3246048]

# Delete all inconclusive results
# Delete CID 3246048, which fails featurization
full_df = full_df[full_df["PUBCHEM_ACTIVITY_OUTCOME"] != "Inconclusive"]

compound_ids = list()
smiles_list = list()
fingerprints = list()
activities = list()

#fingerprint_df = 

for index, row in full_df.iterrows() :
    cid = row["PUBCHEM_CID"]
    smiles_string = row["Smiles"]
    mol = Chem.MolFromSmiles(smiles_string)
    is_active = row["PUBCHEM_ACTIVITY_OUTCOME"] == "Active"
    if mol is None:
        logger.info("Molecule failed featurization")
        logger.info(index)
    else: 
        fingerprint = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol,2,nBits=2048,useChirality=False,
                                                                     useBondTypes=False,useFeatures=False)
        
        # From RDKit documentation
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(fingerprint, arr)
        fingerprint = arr
        
        compound_ids.append(cid)
        smiles_list.append(smiles_string)
        fingerprints.append(fingerprint)
        activities.append(is_active)
    
    if index % 10000 == 0:
        logger.info("Processed index: {0}".format(index))

fingerprints = np.array(fingerprints)
activities = np.array(activities,dtype=int)
        
logger.info("Sampling")

rus = RandomUnderSampler(random_state=global_random_state)
X, y = rus.fit_sample(fingerprints, activities)

logger.info("Processed all, pickling")

#compound_ids_and_features = (compound_ids, smiles_list, fingerprints, activities)

# Pickle the data to save time in the future
with open('data.classification.undersampled.pickle', 'wb') as f:
    pickle.dump((X,y), f, pickle.HIGHEST_PROTOCOL)

2017-09-28 16:14:47,538 - INFO - Processed index: 10000
2017-09-28 16:14:53,127 - INFO - Processed index: 30000
2017-09-28 16:14:56,012 - INFO - Processed index: 40000
2017-09-28 16:14:58,932 - INFO - Processed index: 50000
2017-09-28 16:15:01,937 - INFO - Processed index: 60000
2017-09-28 16:15:07,860 - INFO - Processed index: 80000
2017-09-28 16:15:10,765 - INFO - Processed index: 90000
2017-09-28 16:15:13,823 - INFO - Processed index: 100000
2017-09-28 16:15:16,793 - INFO - Processed index: 110000
2017-09-28 16:15:19,894 - INFO - Processed index: 120000
2017-09-28 16:15:23,044 - INFO - Processed index: 130000
2017-09-28 16:15:26,229 - INFO - Processed index: 140000
2017-09-28 16:15:29,347 - INFO - Processed index: 150000
2017-09-28 16:15:35,636 - INFO - Processed index: 170000
2017-09-28 16:15:48,530 - INFO - Processed index: 210000
2017-09-28 16:15:52,363 - INFO - Processed index: 220000
2017-09-28 16:16:48,529 - INFO - Sampling
2017-09-28 16:16:59,097 - INFO - Compute classes stat

In [5]:
# What is the performance of a dummy classifier on the training set?

from sklearn.dummy import DummyClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.utils.class_weight import compute_class_weight

import pickle

k_fold_splits = 2
global_random_state = 42

with open('data.classification.undersampled.pickle', 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    (X, y) = pickle.load(f)

# print("Number of samples is: {}".format(len(X)))
    
skf = StratifiedKFold(n_splits=k_fold_splits,shuffle=True,random_state=global_random_state)

roc_auc_avg = 0

for train_index, test_index in skf.split(X,y) :

    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]
    
    classifier = DummyClassifier(random_state=global_random_state)
    classifier.fit(X_train,y_train)
    y_pred = classifier.predict(X_test)
    # logger.info(classification_report(y_test, y_pred))
    # What is the AUC-ROC score?
    auc = roc_auc_score(y_test, y_pred, average='macro', sample_weight=None)
    roc_auc_avg = roc_auc_avg + auc
    
roc_auc_avg = roc_auc_avg / k_fold_splits
logger.info("Average roc_auc score of {} folds is: {}".format(k_fold_splits, roc_auc_avg))


2017-09-28 21:42:36,252 - INFO - Average roc_auc score of 2 folds is: 0.4997517224247154


In [6]:
import keras
print(keras.backend.backend())

Using TensorFlow backend.


tensorflow


In [7]:
# What about a deep neural network?
# Sample code from: https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

from keras.models import Sequential
from keras.layers import Dense
from keras import metrics
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.utils.class_weight import compute_class_weight

import pickle

k_fold_splits = 2
global_random_state = 42

with open('data.classification.undersampled.pickle', 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    (X, y) = pickle.load(f)

#class_weights = compute_class_weight('balanced', np.unique(y), y)

    
def create_model() :
    model = Sequential()
    model.add(Dense(2000, input_dim=2048, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
    return model

skf = StratifiedKFold(n_splits=k_fold_splits,shuffle=True,random_state=global_random_state)

roc_auc_avg = 0

for train_index, test_index in skf.split(X,y) :

    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]

    classifier = KerasClassifier(build_fn=create_model, epochs=10, batch_size=100, verbose=1)
    classifier.fit(X_train,y_train)
    y_pred = classifier.predict(X_test)
    logger.info(classification_report(y_test, y_pred))
    # What is the AUC-ROC score?
    auc = roc_auc_score(y_test, y_pred, average='macro', sample_weight=None)
    roc_auc_avg = roc_auc_avg + auc
    
roc_auc_avg = roc_auc_avg / k_fold_splits
logger.info("Average roc_auc score of {} folds is: {}".format(k_fold_splits, roc_auc_avg))


Epoch 1/10
16110/16110 [==============================] - 12s - loss: 0.6026 - acc: 0.6718    
Epoch 2/10
16110/16110 [==============================] - 0s - loss: 0.4362 - acc: 0.7969     
Epoch 3/10
16110/16110 [==============================] - 0s - loss: 0.2291 - acc: 0.9093     
Epoch 4/10
16110/16110 [==============================] - 0s - loss: 0.0957 - acc: 0.9668     
Epoch 5/10
16110/16110 [==============================] - 0s - loss: 0.0464 - acc: 0.9869     
Epoch 6/10
16110/16110 [==============================] - 0s - loss: 0.0233 - acc: 0.9931     
Epoch 7/10
16110/16110 [==============================] - 0s - loss: 0.0207 - acc: 0.9946     
Epoch 8/10
16110/16110 [==============================] - 0s - loss: 0.0185 - acc: 0.9942     
Epoch 9/10
16110/16110 [==============================] - 0s - loss: 0.0145 - acc: 0.9962     
Epoch 10/10
16100/16112 [============================>.] - ETA: 0s2017-09-28 21:43:58,150 - INFO -              precision    recall  f1-score   s

### 